In [1]:
import os
from math_agent_v0 import MathExerciseAnalyzer

In [2]:
analyzer = MathExerciseAnalyzer(model_name="gpt-4o", temperature=0.0)

In [3]:
import uuid
thread_id = str(uuid.uuid4())

# Initialize state with image paths
initial_state = {
    "image_paths": ["/home/elbaz/Bureau/website/agent/images/exo_1/IMG_4698.jpg", "/home/elbaz/Bureau/website/agent/images/exo_1/IMG_4699.jpg"],
    "base64_images": [],
    "raw_analyses": [],
    "structured_analyses": [],
    "combined_analysis": None,
    "exercise": None,
    "error": None,
    "current_image_index": 0
}

# Create config with thread_id for the checkpointer
config = {"configurable": {"thread_id": thread_id}}

# Run the workflow
result = analyzer.workflow.invoke(initial_state, config=config)

In [ ]:
type(result)

In [4]:
result.keys()

dict_keys(['image_paths', 'base64_images', 'raw_analyses', 'structured_analyses', 'combined_analysis', 'exercise', 'error', 'current_image_index'])

In [12]:
result["exercise"].statement

"The problem involves analyzing a function f that belongs to the class C^1(ℝ_+, ℝ_+). It is given that: f'(x)/f(x) ∼ 2/x. The task is to evaluate or analyze the integral: ∫_0^x f."

In [ ]:
current_index = result["current_image_index"]
base64_image = result["base64_images"][current_index]
        
system_prompt = """You are an expert mathematical exercise analyzer. Analyze the handwritten mathematical exercise and extract:

    1. **Statement**: The mathematical problem or question being asked
    2. **Response**: The handwritten solution or answer provided  
    3. **Domain**: The mathematical domain (Algebra, Calculus, Geometry, Trigonometry, Statistics, etc.)
    4. **Level**: The difficulty level (Elementary, Middle School, High School, College, Advanced)

    Guidelines:
    - Be precise in extracting mathematical expressions and symbols
    - Identify mathematical notation correctly (fractions, exponents, integrals, etc.)
    - Determine the appropriate mathematical domain based on the content
    - Assess difficulty level based on mathematical concepts used
    - Provide confidence scores for your analysis
    - Handle both text and mathematical notation in the handwritten content
    - If this appears to be a continuation from a previous page, note that in your analysis
    - If this is a new problem, treat it as such

    Return your analysis in a clear, structured format."""

from langchain_core.messages import HumanMessage, SystemMessage


message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": f"Please analyze this handwritten mathematical exercise (page {current_index + 1} of {len(result['base64_images'])}). Extract the statement, response, domain, and level."
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            }
        }
    ]
)

response = analyzer.llm.invoke([SystemMessage(content=system_prompt), message])

In [ ]:
current_index = result["current_image_index"]
raw_analysis = result["raw_analyses"][current_index]

structure_prompt = """Extract the following information from the analysis and return it as JSON:

{{
    "statement": "extracted problem statement",
    "response": "extracted solution/answer", 
    "domain": "mathematical domain",
    "level": "difficulty level",
    "confidence_score": score between 0 and 1,
    "is_continuation": boolean indicating if this is a continuation from previous page
}}

Analysis text:
{analysis}"""

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

prompt = ChatPromptTemplate.from_template(structure_prompt)
chain = prompt | analyzer.llm | JsonOutputParser()

structured = chain.invoke({"analysis": raw_analysis})

In [ ]:
structured